In [1]:
!pip install sentence-transformers
import pandas as pd
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
import math
import logging
from datetime import datetime
import os
from collections import defaultdict
from datetime import datetime
import torch

     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 1.8MB 16.0MB/s 
     |████████████████████████████████| 1.2MB 43.0MB/s 
     |████████████████████████████████| 890kB 44.5MB/s 
     |████████████████████████████████| 2.9MB 51.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=60f2f854d4cec7fd5d4920527f22be1de94b227d23f45624a8b7906d2b8a4df9
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=4ebf2da5c0be471cf63a246dd5a03a96aaceddef5d9c043af4b9e17c36795a1a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
def divide_into_par(text, threshold=10):
  splits = text.split('\n')
  pars = []
  par = ''
  for split in splits:
    words = split.split(' ')
    if len(words) > 0 and len(words) < threshold:
      if len(par) > 0:
        pars.append(par)
        par = ''
    elif len(words) > threshold:
      par += split
  if len(pars) > 0:
    return pars
  return [text]

In [3]:
wiki = pickle.load( open( '/content/drive/MyDrive/data/wikipedia2.pickle', 'rb' ))
wiki = pd.DataFrame(data=wiki, columns=['id', 'url', 'title', 'text'])
wiki['paragraph'] = wiki['text'].apply(divide_into_par)
wiki.reset_index(drop=True, inplace=True)
wiki = wiki.explode('paragraph')
wiki.reset_index(drop=True, inplace=True)
wiki = wiki.sort_values(by='title')
wiki.reset_index(drop=True, inplace=True)



In [4]:
from sentence_transformers import util
import time
def search(bi_encoder, corpus, corpus_embd,question):
    query = question

    #Encode the query using the bi-encoder and find potentially relevant passages
    start_time = time.time()
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embd, top_k=10)
    hits = hits[0]  # Get the hits for the first query
    end_time = time.time()

    #Output of top-5 hits
    print("Input question:", query)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))
    for hit in hits[0:10]:
        index = hit['corpus_id']
        article = corpus.iloc[index]
        print('\nTitle:', article['title'])
        print('Paragraph:', article['paragraph'])
        print('Url:', article['url'])
        print('Index:', index)
    print("\n\n========\n")

In [5]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('/content/drive/MyDrive/models/cross-encoder/cds-cross-encoder/cds-cross-encoder-bert-base-multilingual-cased-2021-01-24_22-52-29')


In [6]:
def search_with_cros(bi_encoder,cross_encoder, corpus, corpus_embd,question):
    query = question
    passages= wiki['paragraph'].tolist()[:250000]
    #Encode the query using the bi-encoder and find potentially relevant passages
    start_time = time.time()
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embd, top_k=100)
    hits = hits[0]  # Get the hits for the first query

    #Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    #Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    end_time = time.time()

    #Output of top-5 hits
    print("Input question:", query)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))
    for hit in hits[0:10]:
        index = hit['corpus_id']
        article = wiki.iloc[index]
        print('\nTitle:', article['title'])
        print('Paragraph:', article['paragraph'])
        print('Url:', article['url'])
        print('Index:', index)
    print("\n\n========\n")

In [ ]:
def just_search(bi_encoder, corpus_embd, question):
    query = question
    start_time = time.time()
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embd, top_k=10)
    hits = hits[0]
    end_time = time.time()
    return hits

# **Badanie działania modelów na zbiorze 250k paragrafów**

ocena algorytmów odbywa się heurystycznie, bazuje na wskazaniu pozycji na których znajduje się odpowiedź


## **Model nietrenowany**

In [7]:
bi_encoder = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
corpus_embeddings_for_bi_encoder = pickle.load( open( '/content/drive/MyDrive/data/corpus_embeddings_bi_encoder.pickle', 'rb' ))

100%|██████████| 1.01G/1.01G [00:53<00:00, 19.1MB/s]


In [8]:
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"darmowy serwer http")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"środki zwalczające infekcje bakteryjne")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"pierwotniak posiadający nibynóżki")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"substancje pozwalające zabarwić ciasto")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 11.218 seconds):

Title: Cache server
Paragraph: Cache server – sposób organizacji pamięci w systemach klient-serwer, w którym dane pobierane z serwera mogą być przechowywane na serwerze pośredniczącym. Przy kolejnym zapytaniu o te same dane pobierane są one z pamięci podręcznej, co eliminuje nadmierną częstotliwość przesyłań w sieci.
Url: http://pl.wikipedia.org/wiki/?curid=478187
Index: 204260

Title: Bitstream Cyberbit
Paragraph: Bitstream nie oferuje już fontów Bitstream Cyberbit ani jako produkt komercyjny, ani jako darmowe do pobrania do zastosowań niekomercyjnych. Dostępne są one do pobrania z serwera Netscape.
Url: http://pl.wikipedia.org/wiki/?curid=1425913
Index: 159202

Title: BitTorrent
Paragraph: BitTorrent – protokół wymiany i dystrybucji plików przez Internet, którego celem jest odciążenie łączy serwera udostępniającego pliki. Jego największą zaletą w porównaniu do protokołu HTTP jest podział pasma pomiędzy osoby, które 

In [9]:
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"darmowy serwer http")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"środki zwalczające infekcje bakteryjne")
search_with_cros(bi_encoder, cross_encoder,wiki, corpus_embeddings_for_bi_encoder,"pierwotniak posiadający nibynóżki")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search_with_cros(bi_encoder, cross_encoder,wiki, corpus_embeddings_for_bi_encoder,"substancje pozwalające zabarwić ciasto")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 2.293 seconds):

Title: Anelectron
Paragraph: Advanced Electronic Forums (w skrócie AEF) – skrypt forum internetowego. Skrypt jest napisany w języku PHP i wykorzystuje bazę danych MySQL. Udostępniany jest za darmo.
Url: http://pl.wikipedia.org/wiki/?curid=1657123
Index: 87502

Title: Alokacja zasobów
Paragraph: Artykuł zawiera udostępnione na licencji GNU FDL treści pochodzące z serwisu http://mfiles.pl
Url: http://pl.wikipedia.org/wiki/?curid=168154
Index: 71535

Title: Basel II
Paragraph: Artykuł zawiera udostępnione na licencji GNU FDL treści pochodzące z serwisu http://mfiles.pl
Url: http://pl.wikipedia.org/wiki/?curid=1111084
Index: 136264

Title: Cash management
Paragraph: Artykuł zawiera udostępnione na licencji GNU FDL treści pochodzące z serwisu http://mfiles.pl
Url: http://pl.wikipedia.org/wiki/?curid=2238044
Index: 211957

Title: Chmura obliczeniowa
Paragraph: Infrastructure as a Service (z ang. „infrastruktura jako usługa”)

In [10]:
print("")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"co to jest akwarium")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"co to jest akwarela")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"czym jest choinka")
search(bi_encoder, wiki, corpus_embeddings_for_bi_encoder,"do czego służy batyskaf")

print(">>>>>>=========================================================<<<<<<")

search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"co to jest akwarium")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"co to jest akwarela")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"czym jest choinka")
search_with_cros(bi_encoder, cross_encoder, wiki, corpus_embeddings_for_bi_encoder,"do czego służy batyskaf")


Input question: co to jest akwarium
Results (after 0.630 seconds):

Title: Akwarium tarliskowe
Paragraph: Akwarium tarliskowe zwane też hodowlanym lub lęgowym - to osobny zbiornik używany w akwarystyce, najczęściej wykonany ze szkła, o wymiarach: około 25 cm wysokości, 40 cm szerokości i od 40 do 80 cm długości. Do zbiornika lęgowego wpuszcza się ryby złowione z akwarium ogólnego, przystępujące aktualnie do tarła. W zbiorniku rozwija się zapłodniona ikra i wychowuje narybek. Na ogół akwarium nie wymaga oświetlenia, ponieważ ikra wielu gatunków ryb akwariowych jest wrażliwa na światło. Musi być jednak wyposażony w grzałkę. Najlepiej bez podłoża, aby łatwiej było czyścić dno. Rośliny umieszcza się tylko wówczas jeżeli zachodzi taka potrzeba, może to być kępka np. mchu jawajskiego. Parametry wody powinny być jak najbardziej zbliżone do tych, jakie rybom danego gatunku są potrzebne podczas tarła. Wodę filtruje się najczęściej filtrem gąbkowym, aby nie zagrażał narybkowi. W akwarium tarlis

### **Efekty działania dla zapytań (model : bi_encoder):**
* "darmowy serwer http" - **6**
* "środki zwalczające infekcje bakteryjne" - **1,2,9**
* "mikroorganizm posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **3**
* "substancje pozwalające zabarwić ciasto" - **BRAK**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **3**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **8**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **2,10**
* "drwniane, okrągłe, naczynie do prania" - **2**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań z cross-encoderem**
* "darmowy serwer http" - **BRAK**
* "środki zwalczające infekcje bakteryjne" - **BRAK**
* "mikroorganizm posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **9**
* "substancje pozwalające zabarwić ciasto" - **2,6**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **2**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **1**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1,2**
* "drwniane, okrągłe, naczynie do prania" - **1**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań wprost(takich gdzie występuje tytuł artykułu)**
 **bez cross-encodera | z cross encoderem**
* "co to jest akwarium" - **1,2,3,7,8,10 | 5,8,9**
* "jakie piosenki śpiewał Andrzej Rosiewicz" - **1 | 1**
* "co to jest akwarela" - **BRAK | BRAK**
* "czym jest choinka" - **BRAK | 5**
* "do czego służy batyskaf" - **BRAK | 5**


## **Model trenowany na cds**

In [7]:
cds_bi_encoder = SentenceTransformer('/content/drive/MyDrive/models/bi-encoder/cds-bi-encoder/cds-bi-encoder-paraphrase-xlm-r-multilingual-v1-2021-01-24_20-05-32')
corpus_embeddings_for_cds_bi_encoder = pickle.load( open( '/content/drive/MyDrive/data/corpus_embeddings_for_cds_bi_encoder.pickle', 'rb' ))

In [7]:
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"darmowy serwer http")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"środki zwalczające infekcje bakteryjne")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"pierwotniak posiadający nibynóżki")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"substancje pozwalające zabarwić ciasto")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 11.601 seconds):

Title: Cache server
Paragraph: Cache server – sposób organizacji pamięci w systemach klient-serwer, w którym dane pobierane z serwera mogą być przechowywane na serwerze pośredniczącym. Przy kolejnym zapytaniu o te same dane pobierane są one z pamięci podręcznej, co eliminuje nadmierną częstotliwość przesyłań w sieci.
Url: http://pl.wikipedia.org/wiki/?curid=478187
Index: 204260

Title: Bitstream Cyberbit
Paragraph: Bitstream nie oferuje już fontów Bitstream Cyberbit ani jako produkt komercyjny, ani jako darmowe do pobrania do zastosowań niekomercyjnych. Dostępne są one do pobrania z serwera Netscape.
Url: http://pl.wikipedia.org/wiki/?curid=1425913
Index: 159202

Title: Apache HTTP Server
Paragraph: Apache – otwarty serwer HTTP dostępny dla wielu systemów operacyjnych (m.in. UNIX, GNU/Linux, BSD, OS X, Microsoft Windows).
Url: http://pl.wikipedia.org/wiki/?curid=8995
Index: 98930

Title: Anelectron
Paragraph: Advanced

In [11]:
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"darmowy serwer http")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"środki zwalczające infekcje bakteryjne")
search_with_cros(cds_bi_encoder, cross_encoder,wiki, corpus_embeddings_for_cds_bi_encoder,"pierwotniak posiadający nibynóżki")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search_with_cros(cds_bi_encoder, cross_encoder,wiki, corpus_embeddings_for_cds_bi_encoder,"substancje pozwalające zabarwić ciasto")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 3.138 seconds):

Title: Anelectron
Paragraph: Advanced Electronic Forums (w skrócie AEF) – skrypt forum internetowego. Skrypt jest napisany w języku PHP i wykorzystuje bazę danych MySQL. Udostępniany jest za darmo.
Url: http://pl.wikipedia.org/wiki/?curid=1657123
Index: 87502

Title: Apache Tomcat
Paragraph: Apache Tomcat – kontener aplikacji webowych (ang. "web container") rozwijany w ramach projektu Apache.Jako kontener aplikacji jest serwerem, który umożliwia uruchamianie aplikacji internetowych w technologiach java servlets i Java Server Pages ("JSP"). Jest to jeden z bardziej popularnych kontenerów Web. Tomcat jest wykorzystywany w takich serwerach aplikacji JEE (J2EE) jak Apache Geronimo. Jest również bardzo popularnym kontenerem dla samodzielnych aplikacji (nie wymagających pełnego serwera aplikacji) pisanych w środowisku Spring Framework.Apache Tomcat początkowo rozwijany był jako podprojekt projektu Apache Jakarta. W 2005 roku

In [9]:
print("")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"co to jest akwarium")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"co to jest akwarela")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"czym jest choinka")
search(cds_bi_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"do czego służy batyskaf")

print(">>>>>>=========================================================<<<<<<")

search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"co to jest akwarium")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"co to jest akwarela")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"czym jest choinka")
search_with_cros(cds_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_bi_encoder,"do czego służy batyskaf")


Input question: co to jest akwarium
Results (after 0.660 seconds):

Title: Akwarium publiczne
Paragraph: Akwarium publiczne to według kryterium przeznaczenia jedna z trzech form akwarium (obok akwarium domowego oraz akwarium badawczego, przy czym atrybut "publiczne" ma tutaj znaczenie głównie "pokazowe"). Jest to akwarium wystawowe ("exhibition aquarium") będące dużym szklanym zbiornikiem wodnym służącym do publicznych pokazów organizmów wodnych.
Url: http://pl.wikipedia.org/wiki/?curid=2267350
Index: 54856

Title: Akwarium publiczne
Paragraph: Zarówno nazwą "oceanarium", jak i "akwarium" w instytucjonalnym wymiarze określane są całe obiekty wraz z zapleczem technicznym oraz instytucje organizujące ekspozycje organizmów wodnych. Źródła literaturowe zwracają uwagę na następujące różnice: W znaczeniu podstawowym natomiast akwarium (jako zbiornik) jest częścią oceanarium (jako instytucji organizującej ekspozycję).
Url: http://pl.wikipedia.org/wiki/?curid=2267350
Index: 54857

Title: Akwa

### **Efekty działania dla zapytań (model : cds_bi_encoder):**
* "darmowy serwer http" - **3,8**
* "środki zwalczające infekcje bakteryjne" - **1,2,3,4,6,8,10**
* "mikroorganizm posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **1**
* "substancje pozwalające zabarwić ciasto" - **1,3**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **1,3**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **1**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1**
* "drwniane, okrągłe, naczynie do prania" - **7**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań z cross-encoderem**
* "darmowy serwer http" - **2,10**
* "środki zwalczające infekcje bakteryjne" - **1**
* "mikroorganizm posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **6**
* "substancje pozwalające zabarwić ciasto" - **2,9**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **2**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **1**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1,3**
* "drwniane, okrągłe, naczynie do prania" - **1,6**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań wprost(takich gdzie występuje tytuł artykułu)**
 **bez cross-encodera | z cross encoderem**
* "co to jest akwarium" - **1,2,4,6,7 | 5,7,9**
* "jakie piosenki śpiewał Andrzej Rosiewicz" - **1 | 3**
* "co to jest akwarela" - **BRAK | BRAK**
* "czym jest choinka" - **BRAK | 7**
* "do czego służy batyskaf" - **9 | BRAK**


## **Model trenowany na cds i czywiesz**

In [7]:
cds_czywiesz_bi_encoder = SentenceTransformer('/content/drive/MyDrive/models/bi-encoder/cds-czywiesz-bi-encoder/cds-czywiesz-bi-encoder--content-drive-MyDrive-models-cds-bi-encoder-cds-bi-encoder-paraphrase-xlm-r-multilingual-v1-2021-01-24_20-05-32-2021-01-24_22-00-53')
corpus_embeddings_for_cds_czywiesz_bi_encoder = pickle.load( open( '/content/drive/MyDrive/data/corpus_embeddings_for_cds_czywiesz_bi_encoder.pickle', 'rb' ))

In [8]:
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"darmowy serwer http")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"środki zwalczające infekcje bakteryjne")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"pierwotniak posiadający nibynóżki")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"substancje pozwalające zabarwić ciasto")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 12.697 seconds):

Title: ALT Linux
Paragraph: Od wersji 4.0, dostępne jest specjalizowane serwerowe wydanie pod nazwą "Server".System przeznaczony jest zarówno dla architektury x86, jak i dla x86-64.
Url: http://pl.wikipedia.org/wiki/?curid=1557571
Index: 35236

Title: Cache server
Paragraph: Cache server – sposób organizacji pamięci w systemach klient-serwer, w którym dane pobierane z serwera mogą być przechowywane na serwerze pośredniczącym. Przy kolejnym zapytaniu o te same dane pobierane są one z pamięci podręcznej, co eliminuje nadmierną częstotliwość przesyłań w sieci.
Url: http://pl.wikipedia.org/wiki/?curid=478187
Index: 204260

Title: AJAX
Paragraph: Teoretycznie są to wszystkie wymagane elementy, jednak w praktyce używane są jeszcze odpowiednie skrypty funkcjonujące po stronie serwera i współpracujące z bazą danych. Można sobie jednak bez nich poradzić, jeśli wszystkie potrzebne dane zostały już wcześniej wygenerowane (np. za

In [9]:
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"darmowy serwer http")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"środki zwalczające infekcje bakteryjne")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"pierwotniak posiadający nibynóżki")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"substancje pozwalające zabarwić ciasto")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 14.385 seconds):

Title: Anelectron
Paragraph: Advanced Electronic Forums (w skrócie AEF) – skrypt forum internetowego. Skrypt jest napisany w języku PHP i wykorzystuje bazę danych MySQL. Udostępniany jest za darmo.
Url: http://pl.wikipedia.org/wiki/?curid=1657123
Index: 87502

Title: Chmura obliczeniowa
Paragraph: Infrastructure as a Service (z ang. „infrastruktura jako usługa”) – model polegający na dostarczaniu klientowi infrastruktury informatycznej czyli sprzętu, oprogramowania oraz serwisowania. Klient wykupuje na przykład konkretną liczbę serwerów, przestrzeni dyskowej lub określony zasób pamięci i mocy obliczeniowej. Nie oznacza to jednak, że sprzęt fizycznie zostanie zainstalowany w siedzibie klienta. W tym modelu zdarza się, że klient dostarcza usługodawcy własne oprogramowanie do zainstalowania na wynajmowanym sprzęcie.
Url: http://pl.wikipedia.org/wiki/?curid=1518378
Index: 230029

Title: Bitstream Cyberbit
Paragraph: Bitst

In [10]:
print("")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"co to jest akwarium")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"co to jest akwarela")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"czym jest choinka")
search(cds_czywiesz_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"do czego służy batyskaf")

print(">>>>>>=========================================================<<<<<<")

search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"co to jest akwarium")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"co to jest akwarela")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"czym jest choinka")
search_with_cros(cds_czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_bi_encoder,"do czego służy batyskaf")


Input question: co to jest akwarium
Results (after 0.628 seconds):

Title: Akwarium publiczne
Paragraph: Zarówno nazwą "oceanarium", jak i "akwarium" w instytucjonalnym wymiarze określane są całe obiekty wraz z zapleczem technicznym oraz instytucje organizujące ekspozycje organizmów wodnych. Źródła literaturowe zwracają uwagę na następujące różnice: W znaczeniu podstawowym natomiast akwarium (jako zbiornik) jest częścią oceanarium (jako instytucji organizującej ekspozycję).
Url: http://pl.wikipedia.org/wiki/?curid=2267350
Index: 54857

Title: Akwarium publiczne
Paragraph: Akwarium publiczne to według kryterium przeznaczenia jedna z trzech form akwarium (obok akwarium domowego oraz akwarium badawczego, przy czym atrybut "publiczne" ma tutaj znaczenie głównie "pokazowe"). Jest to akwarium wystawowe ("exhibition aquarium") będące dużym szklanym zbiornikiem wodnym służącym do publicznych pokazów organizmów wodnych.
Url: http://pl.wikipedia.org/wiki/?curid=2267350
Index: 54856

Title: Akwa

### **Efekty działania dla zapytań (model : cds_czywiesz_bi_encoder):**
* "darmowy serwer http" - **5,7**
* "środki zwalczające infekcje bakteryjne" - **1,2,3,5,8,9,10**
* "pierwotniak posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **2,3**
* "substancje pozwalające zabarwić ciasto" - **1,5,10**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **1,6,7**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **5**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1,6**
* "drwniane, okrągłe, naczynie do prania" - **BRAK**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań z cross-encoderem**
* "darmowy serwer http" - **4**
* "środki zwalczające infekcje bakteryjne" - **1**
* "mikroorganizm posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **7**
* "substancje pozwalające zabarwić ciasto" - **2,9**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **2**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **1**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1,3**
* "drwniane, okrągłe, naczynie do prania" - **1**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań wprost(takich gdzie występuje tytuł artykułu)**
 **bez cross-encodera | z cross encoderem**
* "co to jest akwarium" - **1,2,4,6,7,8,10 | 3,6,9**
* "jakie piosenki śpiewał Andrzej Rosiewicz" - **1 | 2**
* "co to jest akwarela" - **BRAK | BRAK**
* "czym jest choinka" - **1,3 | BRAK**
* "do czego służy batyskaf" - **4 | BRAK**

## **Model trenowany na cds, czywiesz i labels**

labels - nasz własny zbiór stworzony na podstawie danych zbioru czywiesz; pary pytanie - odpowiedz zostały zaetykietowane



In [7]:
cds_czywiesz_label_bi_encoder = SentenceTransformer('/content/drive/MyDrive/models/bi-encoder/cds-czywiesz-labels-bi-encoder/cds_czywiesz_labels-bi-encoder-bert-base-multilingual-cased-2021-01-24_22-52-29')
corpus_embeddings_for_cds_czywiesz_label_bi_encoder = pickle.load( open( '/content/drive/MyDrive/data/corpus_embeddings_for_cds_czywiesz_labels_bi_encoder.pickle', 'rb' ))

In [7]:
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"darmowy serwer http")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"środki zwalczające infekcje bakteryjne")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"pierwotniak posiadający nibynóżki")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"substancje pozwalające zabarwić ciasto")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 11.457 seconds):

Title: Anelectron
Paragraph: Advanced Electronic Forums (w skrócie AEF) – skrypt forum internetowego. Skrypt jest napisany w języku PHP i wykorzystuje bazę danych MySQL. Udostępniany jest za darmo.
Url: http://pl.wikipedia.org/wiki/?curid=1657123
Index: 87502

Title: Cache server
Paragraph: Cache server – sposób organizacji pamięci w systemach klient-serwer, w którym dane pobierane z serwera mogą być przechowywane na serwerze pośredniczącym. Przy kolejnym zapytaniu o te same dane pobierane są one z pamięci podręcznej, co eliminuje nadmierną częstotliwość przesyłań w sieci.
Url: http://pl.wikipedia.org/wiki/?curid=478187
Index: 204260

Title: Bitstream Cyberbit
Paragraph: Bitstream nie oferuje już fontów Bitstream Cyberbit ani jako produkt komercyjny, ani jako darmowe do pobrania do zastosowań niekomercyjnych. Dostępne są one do pobrania z serwera Netscape.
Url: http://pl.wikipedia.org/wiki/?curid=1425913
Index: 159202

In [8]:
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"darmowy serwer http")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"środki zwalczające infekcje bakteryjne")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"pierwotniak posiadający nibynóżki")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"substancje pozwalające zabarwić ciasto")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 13.123 seconds):

Title: Bridge Base Online
Paragraph: Bridge Base Online (BBO) - darmowy serwis internetowy umożliwiający grę w brydża online.
Url: http://pl.wikipedia.org/wiki/?curid=2152650
Index: 185909

Title: Blog.pl
Paragraph: Blog.pl – polski serwis blogowy, założony w 2001 roku przez Adama Wojtkiewicza oraz Bartłomieja Pogodę. Początkowo korzystanie z jego usług było częściowo odpłatne. Od grudnia 2006 roku partnerem serwisu jest portal Onet.pl, korzystanie z niego jest darmowe; aby założyć blog, wystarczy mieć konto pocztowe. System umożliwia otwartą edycję kodu HTML. W grudniu 2007 roku wraz z odświeżoną wersją serwisu pojawiło się centrum debat – system otwarty dla wszystkich blogerów, bez względu na platformę, na której piszą, umożliwiający wymianę poglądów na aktualne tematy. Dysputy prowadzone są na poszczególnych blogach, a w serwisie pojawiają się jedynie ich skróty odsyłające do blogu autora. Uzupełnieniem mechanizmu 

In [8]:
print("")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"co to jest akwarium")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"co to jest akwarela")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"czym jest choinka")
search(cds_czywiesz_label_bi_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"do czego służy batyskaf")

print(">>>>>>=========================================================<<<<<<")

search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"co to jest akwarium")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"co to jest akwarela")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"czym jest choinka")
search_with_cros(cds_czywiesz_label_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_cds_czywiesz_label_bi_encoder,"do czego służy batyskaf")


Input question: co to jest akwarium
Results (after 14.943 seconds):

Title: Akwarium publiczne
Paragraph: Akwarium publiczne to według kryterium przeznaczenia jedna z trzech form akwarium (obok akwarium domowego oraz akwarium badawczego, przy czym atrybut "publiczne" ma tutaj znaczenie głównie "pokazowe"). Jest to akwarium wystawowe ("exhibition aquarium") będące dużym szklanym zbiornikiem wodnym służącym do publicznych pokazów organizmów wodnych.
Url: http://pl.wikipedia.org/wiki/?curid=2267350
Index: 54856

Title: Akwarium holenderskie
Paragraph: Akwarium holenderskie - dekoracyjne akwarium przeznaczone głównie do hodowli roślin akwariowych. Wyróżniające się niewielką obsadą lub brakiem ryb i staranną kompozycją dobranych roślin akwariowych.Pierwsze tego typu akwaria zostały szeroko zaprezentowane na wystawach akwarystycznych przez akwarystów holenderskich - stąd pochodzi ich zwyczajowa nazwa.Wyposażone w wysokiej klasy sprzęt oświetleniowy spełniający wymagania świetlne roślin. Sto

### **Efekty działania dla zapytań (model : cds_czywiesz_label_bi_encoder):**
* "darmowy serwer http" - **7**
* "środki zwalczające infekcje bakteryjne" - **1,2,4,5,9**
* "pierwotniak posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **5**
* "substancje pozwalające zabarwić ciasto" - **5**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **1**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **2**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1**
* "drwniane, okrągłe, naczynie do prania" - **BRAK**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań z cross-encoderem**
* "darmowy serwer http" - **4**
* "środki zwalczające infekcje bakteryjne" - **1**
* "mikroorganizm posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **BRAK**
* "substancje pozwalające zabarwić ciasto" - **2**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **2**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **1**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1,4**
* "drwniane, okrągłe, naczynie do prania" - **1**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań wprost(takich gdzie występuje tytuł artykułu)**
 **bez cross-encodera | z cross encoderem**
* "co to jest akwarium" - **1,2,3,4,5,10 | 4,8,10**
* "jakie piosenki śpiewał Andrzej Rosiewicz" - **1 | 2**
* "co to jest akwarela" - **BRAK | BRAK**
* "czym jest choinka" - **BRAK | 6**
* "do czego służy batyskaf" - **4 | BRAK**

## **Model trenowany na czywiesz**

In [6]:
czywiesz_bi_encoder = SentenceTransformer('/content/drive/MyDrive/models/bi-encoder/czywiesz-bi-encoder/czywiesz-bi-encoder-paraphrase-xlm-r-multilingual-v1-2021-01-24_21-20-22')
corpus_embeddings_for_czywiesz_bi_encoder = pickle.load( open( '/content/drive/MyDrive/data/corpus_embeddings_for_czywiesz_bi_encoder.pickle', 'rb' ))

In [8]:
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"darmowy serwer http")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"środki zwalczające infekcje bakteryjne")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"pierwotniak posiadający nibynóżki")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"substancje pozwalające zabarwić ciasto")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 11.665 seconds):

Title: Cache server
Paragraph: Cache server – sposób organizacji pamięci w systemach klient-serwer, w którym dane pobierane z serwera mogą być przechowywane na serwerze pośredniczącym. Przy kolejnym zapytaniu o te same dane pobierane są one z pamięci podręcznej, co eliminuje nadmierną częstotliwość przesyłań w sieci.
Url: http://pl.wikipedia.org/wiki/?curid=478187
Index: 204260

Title: AJAX
Paragraph: Teoretycznie są to wszystkie wymagane elementy, jednak w praktyce używane są jeszcze odpowiednie skrypty funkcjonujące po stronie serwera i współpracujące z bazą danych. Można sobie jednak bez nich poradzić, jeśli wszystkie potrzebne dane zostały już wcześniej wygenerowane (np. zawartość poszczególnych stron prostego serwisu).
Url: http://pl.wikipedia.org/wiki/?curid=186916
Index: 35095

Title: CesarFTP
Paragraph: CesarFTP – darmowy program typu Serwer FTP. Nie posiada on swojej wersji klienckiej (klienta FTP tego samego

In [9]:
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"darmowy serwer http")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"środki zwalczające infekcje bakteryjne")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"pierwotniak posiadający nibynóżki")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"najmniejsze żywe organizmy na ziemi")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"substancje pozwalające zabarwić ciasto")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"konstrukcja zapobiegająca wypadnięciu z drogi")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"tradycyjne polskie danie z kapusty, mięsa i grzybów")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"drwniane, okrągłe, naczynie do prania")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"kto śpiewa piosenkę w serialu czterdziestolatek")

Input question: darmowy serwer http
Results (after 3.645 seconds):

Title: Blog.pl
Paragraph: Blog.pl – polski serwis blogowy, założony w 2001 roku przez Adama Wojtkiewicza oraz Bartłomieja Pogodę. Początkowo korzystanie z jego usług było częściowo odpłatne. Od grudnia 2006 roku partnerem serwisu jest portal Onet.pl, korzystanie z niego jest darmowe; aby założyć blog, wystarczy mieć konto pocztowe. System umożliwia otwartą edycję kodu HTML. W grudniu 2007 roku wraz z odświeżoną wersją serwisu pojawiło się centrum debat – system otwarty dla wszystkich blogerów, bez względu na platformę, na której piszą, umożliwiający wymianę poglądów na aktualne tematy. Dysputy prowadzone są na poszczególnych blogach, a w serwisie pojawiają się jedynie ich skróty odsyłające do blogu autora. Uzupełnieniem mechanizmu debat jest blogometr – narzędzie obrazujące rozkład poglądów blogerów. W efekcie prowadzonych zmian w serwisie zostały wyraźnie rozdzielone dwie przestrzenie: międzyblogowych debat oraz blogó

In [9]:
print("")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"co to jest akwarium")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"co to jest akwarela")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"czym jest choinka")
search(czywiesz_bi_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"do czego służy batyskaf")

print(">>>>>>=========================================================<<<<<<")

search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"co to jest akwarium")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"jakie piosenki śpiewał Andrzej Rosiewicz")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"co to jest akwarela")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"czym jest choinka")
search_with_cros(czywiesz_bi_encoder, cross_encoder, wiki, corpus_embeddings_for_czywiesz_bi_encoder,"do czego służy batyskaf")


Input question: co to jest akwarium
Results (after 0.906 seconds):

Title: Akwarium publiczne
Paragraph: Zarówno nazwą "oceanarium", jak i "akwarium" w instytucjonalnym wymiarze określane są całe obiekty wraz z zapleczem technicznym oraz instytucje organizujące ekspozycje organizmów wodnych. Źródła literaturowe zwracają uwagę na następujące różnice: W znaczeniu podstawowym natomiast akwarium (jako zbiornik) jest częścią oceanarium (jako instytucji organizującej ekspozycję).
Url: http://pl.wikipedia.org/wiki/?curid=2267350
Index: 54857

Title: Akwarium
Paragraph: Akwarium (łac. "aquarium" – naczynie na wodę) – sztuczny zbiornik wodny o zróżnicowanych rozmiarach i kształtach, z co najmniej jedną ścianą z materiału przezroczystego, zwykle ze szkła, służący do hodowli oraz prezentacji organizmów żyjących w środowisku słodko- lub słonowodnym. Termin "akwarium" jest także stosowany w odniesieniu do wycinka zawartego w takim zbiorniku środowiska wodnego obejmującego organizmy wodne, utrzymyw

### **Efekty działania dla zapytań (model : czywiesz_bi_encoder):**
* "darmowy serwer http" - **6,7**
* "środki zwalczające infekcje bakteryjne" - **1,3,4,8**
* "pierwotniak posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **1,10**
* "substancje pozwalające zabarwić ciasto" - **4**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **1,2,5,6,7,10**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **6**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1**
* "drwniane, okrągłe, naczynie do prania" - **9**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań z cross-encoderem**
* "darmowy serwer http" - **3,6**
* "środki zwalczające infekcje bakteryjne" - **1**
* "mikroorganizm posiadający nibynóżki" - **BRAK**
* "najmniejsze żywe organizmy na ziemi" - **5**
* "substancje pozwalające zabarwić ciasto" - **2,7**
* "konstrukcja zapobiegająca wypadnięciu z drogi" - **1**
* "powlekana tkanina, wytrzymała mechanicznie, do stosowania w kuchni" - **1**
* "tradycyjne polskie danie z kapusty, mięsa i grzybów" - **1,3,4**
* "drwniane, okrągłe, naczynie do prania" - **1**
* "kto śpiewa piosenkę w serialu czterdziestolatek" - **BRAK**

#### **Efekty działania dla zapytań wprost(takich gdzie występuje tytuł artykułu)**
 **bez cross-encodera | z cross encoderem**
* "co to jest akwarium" - **1,2,3,4,5,8,10 | 5**
* "jakie piosenki śpiewał Andrzej Rosiewicz" - **1 | 1**
* "co to jest akwarela" - **BRAK | BRAK**
* "czym jest choinka" - **BRAK | 2,3,4**
* "do czego służy batyskaf" - **2 | BRAK**